|<h2>Course:</h2>|<h1><a href="https://udemy.com/course/dullms_x/?couponCode=202508" target="_blank">A deep understanding of AI language model mechanisms</a></h1>|
|-|:-:|
|<h2>Part 5:</h2>|<h1>Observation (non-causal) mech interp<h1>|
|<h2>Section:</h2>|<h1>Investigating layers<h1>|
|<h2>Lecture:</h2>|<h1><b>CodeChallenge: Logit Lens in BERT<b></h1>|

<br>

<h5><b>Teacher:</b> Mike X Cohen, <a href="https://sincxpress.com" target="_blank">sincxpress.com</a></h5>
<h5><b>Course URL:</b> <a href="https://udemy.com/course/dullms_x/?couponCode=202508" target="_blank">udemy.com/course/dullms_x/?couponCode=202508</a></h5>
<i>Using the code without the course may lead to confusion or errors.</i>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.gridspec import GridSpec

import torch
import torch.nn.functional as F

import matplotlib_inline.backend_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('svg')

# Exercise 1: Reminder of masked-token predictions in BERT

In [ ]:
from transformers import BertTokenizer, BertForMaskedLM

# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')
model = BertForMaskedLM.from_pretrained('bert-large-uncased')

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
model.eval()

In [ ]:
# getting predicts in BERT (note: target word for [MASK] is "way")
text = 'The way you do anything is the [MASK] you do everything.'
tokens = tokenizer.encode(text, return_tensors='pt')

for t in tokens[0]:
  print(f'Token {t:5} is "{tokenizer.decode(t)}"')

# index of the [MASK] token
mask_token_idx = torch.where(tokens == tokenizer.mask_token_id)[1]
print(f'\nMask token is {tokenizer.mask_token_id} and is in index {mask_token_idx.item()}')

In [ ]:
# forward pass
with torch.no_grad():
  outputs = model(tokens.to(device))

# logits for [MASK] token
mask_logits = outputs.logits[0,mask_token_idx,:].squeeze().cpu()

# top predicted token
top_token_idx = torch.argmax(mask_logits, dim=-1)

# and plot
plt.figure(figsize=(10,4))

plt.plot(top_token_idx,outputs.logits[0,mask_token_idx,top_token_idx].cpu(),'go',markersize=9)
plt.plot(mask_logits,'k.',alpha=.3)
plt.gca().set(xlim=[-5,tokenizer.vocab_size+4],xlabel='Token index',ylabel='BERT output logits',
              title=f'Predicted token is "{tokenizer.decode(top_token_idx)}"')
plt.show()

# Exercise 2: Calculate and visualize z-scores

In [ ]:
# calculate z
zLogits = (mask_logits-mask_logits.mean()) / mask_logits.std()

# and plot
fig = plt.figure(figsize=(13,4))
gs = GridSpec(1,3,figure=fig)

ax0 = fig.add_subplot(gs[0])
ax1 = fig.add_subplot(gs[1:])

# histogram of final output layer logits
ax0.hist(mask_logits.numpy(),bins=80,edgecolor='k',facecolor='gray',label='All tokens')
ax0.axvline(mask_logits[top_token_idx],linestyle='--',color='g',label='Predicted token')
ax0.set(xlabel='Logit value',ylabel='Count',title='Histogram of logits')
ax0.legend(fontsize=9)

# scatter plot of logits
ax1.plot(top_token_idx,zLogits[top_token_idx],'go',markersize=9)
ax1.plot(zLogits,'k.',alpha=.3)
ax1.set(xlim=[-5,tokenizer.vocab_size+4],xlabel='Token index',ylabel='Z-scored output logits',
              title=f'Predicted token is "{tokenizer.decode(top_token_idx)}"')

plt.tight_layout()
plt.show()

# Exercise 3: Z-score final output logits for each token

In [ ]:
# full sentence
text = 'the way you do anything is the way you do everything'
tokens = tokenizer.encode(text,return_tensors='pt')

# initialize
mask_zscores = np.zeros(len(tokens[0]))
predicted_text = []

print(f'[MASK] token is {tokenizer.mask_token_id}\n')

# loop over tokens, replace with [MASK], and get logits
for idx,tok in enumerate(tokens[0]):

  # make a copy and replace a token with mask
  masked_tokens = tokens.clone().to(device)
  masked_tokens[0,idx] = tokenizer.mask_token_id
  # confirmation (convert to list for visualizability):
  print(masked_tokens[0].tolist())

  # forward pass through the model
  with torch.no_grad(): outputs = model(masked_tokens)

  # get logits for the masked position
  mask_logits = outputs.logits[0,idx,:].squeeze().cpu()

  # get the max masked prediction and its z-score
  predicted_token = torch.argmax(mask_logits,dim=-1)
  predicted_text.append( tokenizer.decode(predicted_token) )
  mask_zscores[idx] = (mask_logits[predicted_token]-mask_logits.mean()) / mask_logits.std()

In [ ]:
# print a table of results

print('    TARGET   |  PREDICTED |  Z-SCORE')
print('-'*38)
for idx in range(len(tokens[0])):
  print(f'{tokenizer.decode(tokens[0,idx]):>10}   |   {predicted_text[idx]:^6}   |   {mask_zscores[idx]:5.2f}')


# Exercise 4: Proper internal lensing in BERT

In [ ]:
# reminder
model

In [ ]:
# convenience variable (+1 for initial embedding layer)
numlayers = len(model.bert.encoder.layer) + 1

In [ ]:
# getting predicts in BERT (note: target word for [MASK] is "way")
text_withMask = 'The way you do anything is the [MASK] you do everything.'
masked_tokens = tokenizer.encode(text_withMask, return_tensors='pt').to(device)

In [ ]:
with torch.no_grad(): outputs = model(masked_tokens,output_hidden_states=True)
print(f'There are {numlayers} layers')
outputs.hidden_states[4].shape

In [ ]:
### THE WRONG WAY

# unembedding matrix
unembeddingT = model.cls.predictions.decoder.weight.detach().T

# activations from mask position in one layer
acts = outputs.hidden_states[-3][0,8,:].detach()

# same as with GPT?
mask_logitsWrong = acts @ unembeddingT

mask_logitsWrong = mask_logitsWrong.cpu()

In [ ]:
### THE CORRECT WAY
mask_logitsCorrect = model.cls.predictions(acts).detach().cpu()

In [ ]:
# predictions
_,axs = plt.subplots(1,2,figsize=(14,4))

# the WRONG way
top_token_idx_wrong = torch.argmax(mask_logitsWrong,dim=-1).cpu()

axs[0].plot(top_token_idx_wrong,mask_logitsWrong[top_token_idx_wrong],'go',markersize=9)
axs[0].plot(mask_logitsWrong,'k.',alpha=.3)
axs[0].set(xlim=[-10,tokenizer.vocab_size+9],xlabel='Token index',ylabel='Z-scored output logits',
              title=f'(WRONG WAY) Predicted token is "{tokenizer.decode(top_token_idx_wrong)}"')


# the CORRECT way
top_token_idx_correct = torch.argmax(mask_logitsCorrect,dim=-1)

axs[1].plot(top_token_idx_correct,mask_logitsCorrect[top_token_idx_correct],'go',markersize=9)
axs[1].plot(mask_logitsCorrect,'k.',alpha=.3)
axs[1].set(xlim=[-10,tokenizer.vocab_size+9],xlabel='Token index',ylabel='Z-scored output logits',
              title=f'(CORRECT WAY) Predicted token is "{tokenizer.decode(top_token_idx_correct)}"')

plt.tight_layout()
plt.show()

# Exercise 5: Repeat for all layers (logit-lens)

In [ ]:
# initialize
mask_zscores = np.zeros((numlayers,len(tokens[0])))
predictedTokens = np.zeros((numlayers,len(tokens[0])),dtype=int)

# loop over tokens, replace with [MASK], and get logits
for idx,tok in enumerate(tokens[0]):

  # make a copy and replace a token with mask
  masked_tokens = tokens.clone()
  masked_tokens[0,idx] = tokenizer.mask_token_id
  # confirmation:
  #print(masked_tokens[0])

  # forward pass through the model (with hidden states exported!)
  with torch.no_grad():
    outputs = model(masked_tokens.to(device),output_hidden_states=True)


  ### loop over layers
  for layeri in range(numlayers):

    # get internal logits for the masked position
    acts = outputs.hidden_states[layeri][0,idx,:]

    # logit lens
    mask_logits = model.cls.predictions(acts).cpu()

    # get the masked prediction and its z-score
    predicted_token = torch.argmax(mask_logits,dim=-1)
    predictedTokens[layeri,idx] = predicted_token
    mask_zscores[layeri,idx] = (mask_logits[predicted_token]-mask_logits.mean()) / mask_logits.std()

In [ ]:
print('Original text:\n', text, '\n')
print('Predictions at first transformer block:\n', tokenizer.decode(predictedTokens[1,1:-1]), '\n')
print('Predictions at final transformer block:\n', tokenizer.decode(predictedTokens[-1,1:-1]))

In [ ]:
# lines visualization
plt.figure(figsize=(12,4))

for layeri in range(numlayers):
  plt.plot(range(len(tokens[0])-2),mask_zscores[layeri,1:-1].T,'o-',color=mpl.cm.plasma((layeri+1)/numlayers),label=f'L{layeri}')

plt.gca().set(xticks=range(len(tokens[0,1:-1])),xticklabels=[tokenizer.decode(t) for t in tokens[0,1:-1]],
              ylabel='Z-score',title='Z-scores of predicted tokens')

plt.legend(fontsize=11.3,bbox_to_anchor=(1,1.03),ncol=2)
plt.show()

In [ ]:
# and as an image
plt.figure(figsize=(13,5))
plt.imshow(mask_zscores[:,1:-1],aspect='auto',origin='lower',cmap='plasma')
plt.colorbar(pad=.01,label='Z-score')

plt.gca().set(xlabel='Target word',ylabel='Layer',
              xticks=range(len(tokens[0,1:-1])),xticklabels=[tokenizer.decode(t) for t in tokens[0,1:-1]])
plt.show()

# Exercise 6: Logit Lens text heatmap

In [ ]:
# scale z-scores
scaled_zscores = (mask_zscores-mask_zscores.min()) / (mask_zscores.max()-mask_zscores.min())
np.sort(scaled_zscores.flatten())

In [ ]:
fig,ax = plt.subplots(1,figsize=(13,10))

# original text (separated into a list of decoded tokens)
target = [tokenizer.decode(t) for t in tokens[0]]
numTokens = len(target)

# loop over layers
for layeri in range(numlayers):

  # y-axis coordinate for this layer
  yCoord = 1-layeri/numlayers

  # print the layer number in the left margin
  ax.text(-.1,yCoord,f'Layer {layeri}:',ha='right')

  # loop over the predicted tokens in this layer
  for xi,tok in enumerate(predictedTokens[layeri]):
    ax.text(xi/numTokens,yCoord,tokenizer.decode(tok),ha='center',
            bbox=dict(boxstyle='round,pad=0.3', facecolor=mpl.cm.Reds(scaled_zscores[layeri,xi]), edgecolor='none',alpha=.4))

ax.axis('off')

# finally, draw the target tokens at the bottom
ax.text(-.1,yCoord-.05,f'Target:',ha='right',fontweight='bold')
for xi,tok in enumerate(target):
  ax.text(xi/numTokens,yCoord-.05,tok,ha='center',fontsize=12,fontweight='bold')

plt.show()